# NG Data Portal API Wrapper Development

<br>

This notebook documents the development of an API wrapper for the National Grid Data Portal. 

It starts by looking at a single data stream, containing the percentage generation of each fuel, moving from basic requests to more developed and generalisable queries. During this process potential areas for user error are identified and warnings put in place to help guide them in their usage. Finally the groups of functions are combined into a single wrapper class within the NGDataPortal script.

The next stage is to identify the different data streams and create a mapping to their resource ids.

<br>

### Imports

In [35]:
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup

import warnings
from itertools import islice

from IPython.core.display import display, HTML

from NGDataPortal import Wrapper, stream_to_id_map

<br>

### Helper Functions

In [2]:
def take(iterable, n=5):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

<br>

### Retrieving Package Information

We can retrieve information relating to specific data streams through their respective datapackage.jsons

In [3]:
form_package_url = lambda group, stream: f'https://national-grid.ckan.io/{group}/{stream}/datapackage.json'

group = 'generation'
stream = 'generation-mix-national'

package_url = form_package_url(group, stream)
r_package = requests.get(package_url)

r_package

<Response [200]>

<br>

These include general information about the data stream

In [4]:
package_desc = r_package.json()['resources'][0]['descriptionHtml']

display(HTML(package_desc))

<br>

As well as important attributes such as the resource id, with this we can now access the API data feeds

In [4]:
resource_id = r_package.json()['resources'][0]['id']

resource_id

'0a168493-5d67-4a26-8344-2fe0a5d4d20b'

<br>

### Creating the API Wrapper

We start the API wrapper development by creating a function which constructs the url for the relevant endpoint and then makes a request to it.

In [25]:
def NG_request(resource_id, params={}):    
    url_root = 'https://national-grid-admin.ckan.io/api/3/action/datastore_search'
    
    params.update({'resource_id':resource_id})
    
    if 'sql' in params.keys():
        url_root += '_sql'
        
    r = requests.get(url_root, params=params)
    
    return r

resource_id = 'f0933bdd-1b0e-4dd3-aa7f-5498df1ba5b9'

r = NG_request(resource_id)
df = pd.DataFrame(r.json()['result']['records'])

df.head(2)

,_id,dtm,f
0,1,2019-12-01T00:00:00,50.118
1,2,2019-12-01T00:00:01,50.117


<br>

Before we take this further we want to add some checks which confirm the call was successful and if not exposes the error message returned by the API. We create a wrapper for the error to enable its assignment as a dictionary value in the next section.

In [18]:
def raise_(err_txt, error=ValueError): 
    raise error(err_txt)
    
def check_request_success(r_json):
    if r_json['success'] == False:
        err_msg = r_json['error']['message']
        raise_(err_msg)
        
r = NG_request('incorrect_resource_id')
r_json = r.json()

#check_request_success(r_json)

<br>

The most common 'advanced' query to the API will include some specification around the date range for the data, within CKAN we can use SQL to construct these specific requests.

In [29]:
date_between = lambda dt_col, resource_id, start_date, end_date: f'SELECT * from "{resource_id}" WHERE "{dt_col}" BETWEEN \'{start_date}\'::timestamp AND \'{end_date}\'::timestamp ORDER BY "{dt_col}"' 
date_less_than = lambda dt_col, resource_id, date: f'SELECT * from "{resource_id}" WHERE "{dt_col}" < \'{date}\'::timestamp ORDER BY "{dt_col}"' 
date_greater_than = lambda dt_col, resource_id, date: f'SELECT * from "{resource_id}" WHERE "{dt_col}" > \'{date}\'::timestamp ORDER BY "{dt_col}"' 

def form_dt_rng_sql_query(dt_col, resource_id, start_date=None, end_date=None):
    start_end_date_exist = (start_date!=None, end_date!=None)
    
    func_map = {
        (False, False) : {'error' : 'A start and/or end date should be passed'},
        (True, True) : date_between(dt_col, resource_id, start_date, end_date),
        (False, True) : date_less_than(dt_col, resource_id, end_date),
        (True, False) : date_greater_than(dt_col, resource_id, start_date),
    }
    
    sql = func_map[start_end_date_exist]
    
    if not isinstance(sql, str):
        raise_(sql['error'])
    
    return sql
    
dt_col = 'dtm'
start_date = '2019-12-01T00:00:00'
end_date = '2019-12-01T00:00:30'

sql = form_dt_rng_sql_query(dt_col, resource_id, start_date=start_date, end_date=end_date)

print(sql)

SELECT * from "f0933bdd-1b0e-4dd3-aa7f-5498df1ba5b9" WHERE "dtm" BETWEEN '2019-12-01T00:00:00'::timestamp AND '2019-12-01T00:00:30'::timestamp ORDER BY "dtm"


<br>

We're now ready to put this all together into a single function

In [32]:
def query_API(resource_id, params={}, start_date=None, end_date=None, dt_col=None, sql=None, return_raw=False):
    ## Handling SQL queries
    if start_date or end_date:
        if sql:
            warnings.warn('The start and end date query will overwrite the provided SQL')
        
        if not dt_col:
            raise_('If a start or end date has been provided the \'dt_col\' parameter must be provided')
        
        sql = form_dt_rng_sql_query(dt_col, resource_id, start_date=start_date, end_date=end_date)
        params.update({'sql':sql})
    
    elif sql:
        params.update({'sql':sql})
    
    ## Making the request
    r = NG_request(resource_id, params=params)
    
    if return_raw == True:
        return r

    ## Checking and parsing the response
    r_json = r.json()
    check_request_success(r_json)
    
    df = pd.DataFrame(r_json['result']['records'])
        
    return df

start_date = '2019-12-01T00:00:00'
end_date = '2019-12-01T00:00:30'
dt_col = 'dtm'

df = query_API(resource_id, start_date=start_date, end_date=end_date, dt_col=dt_col)

df.head()

,_id,_full_text,dtm,f
0,1,"'-01':3 '-12':2 '00':5,6 '2019':1 '50.118':7 '...",2019-12-01T00:00:00,50.118
1,2,'-01':3 '-12':2 '00':5 '01':6 '2019':1 '50.117...,2019-12-01T00:00:01,50.117
2,3,'-01':3 '-12':2 '00':5 '02':6 '2019':1 '50.111...,2019-12-01T00:00:02,50.111
3,4,'-01':3 '-12':2 '00':5 '03':6 '2019':1 '50.1':...,2019-12-01T00:00:03,50.1
4,5,'-01':3 '-12':2 '00':5 '04':6 '2019':1 '50.089...,2019-12-01T00:00:04,50.089


<br>

### Collecting Metadata for Available Streams

In [10]:
url_search_NGDataPortal = lambda q, from_: f'https://data.nationalgrideso.com/search?q={q}&from={from_}&sort=score:desc'

search_url = url_search_NGDataPortal('', 0)
r = requests.get(search_url)

r

<Response [200]>

In [11]:
def response_2_resource_links(r):
    soup = BeautifulSoup(r.content)

    resource_links = soup.findAll('a', {'class':'text-orange hover:text-gray-800'})
    resource_links = [resource_link['href'] for resource_link in resource_links]

    return resource_links

resource_links = response_2_resource_links(r)

resource_links

['/plans-reports-analysis/forward-plan-role-1-tracker-19-20',
 '/system/system-excursions',
 '/ancillary-services/fast-reserve-tender-reports',
 '/system/system-frequency-data',
 '/ancillary-services/firm-frequency-response-post-tender-reports',
 '/ancillary-services/short-term-operating-reserve-stor',
 '/ancillary-services/short-term-operating-reserve-tender-reports',
 '/trade-data/contracted-energy-volumes-and-data',
 '/ancillary-services/obligatory-reactive-power-service-orps-utilisation',
 '/ancillary-services/fast-reserve-market-information-reports']

In [12]:
resource_links = []
num_page_resource_links = 1

while num_page_resource_links > 0:
    search_url = url_search_NGDataPortal('', len(resource_links))
    r = requests.get(search_url)
    
    page_resource_links = response_2_resource_links(r)
    num_page_resource_links = len(page_resource_links)
    
    resource_links += page_resource_links
    
print(f'There are a total of {len(resource_links)} data streams available through the National Grid portal')

df_resources = (pd.Series(resource_links)
                .str[1:]
                .str.split('/')
                .apply(pd.Series)
                .rename(columns={0:'group', 1:'stream'})
               )

df_resources.head()

There are a total of 46 data streams available through the National Grid portal


,group,stream
0,plans-reports-analysis,forward-plan-role-1-tracker-19-20
1,system,system-excursions
2,ancillary-services,fast-reserve-tender-reports
3,system,system-frequency-data
4,ancillary-services,firm-frequency-response-post-tender-reports


In [13]:
def package_url_2_metadata(package_url):
    r_package = requests.get(package_url)
    r_json = r_package.json()

    package_metadata = dict()

    package_metadata['package_desc'] = r_json['resources'][0]['description']
    package_metadata['resource_id'] = r_json['resources'][0]['id']

    return package_metadata

group = 'generation'
stream = 'generation-mix-national'

package_url = form_package_url(group, stream)
package_metadata = package_url_2_metadata(package_url)

package_metadata

{'package_desc': 'This resource provides a rolling 30 day history of the national generation mix by settlement period and is updated every 30 minutes. <br/><br/>\r\n\r\nDatetime is in ISO8601 format YYYY-MM-DDThh:mmZ e.g. 2017-08-25T12:30Z. All times provided in UTC (+00:00).',
 'resource_id': '0a168493-5d67-4a26-8344-2fe0a5d4d20b'}

In [14]:
%%time

identify_resource_ids = False

if identify_resource_ids == True:
    for idx, (group, stream) in df_resources.iterrows():
        package_url = form_package_url(group, stream)
        package_metadata = package_url_2_metadata(package_url)

        df_resources.loc[idx, 'resource_id'] = package_metadata['resource_id']

    stream_2_id_map = df_resources.set_index('stream')['resource_id'].to_dict()

    print(dict(take(stream_2_id_map.items())))

Wall time: 0 ns


In [15]:
if identify_resource_ids == True:
    with open('stream_to_resource_id.json', 'w') as fp:
        json.dump(stream_2_id_map, fp)

In [43]:
stream = 'current-balancing-services-use-of-system-bsuos-data'
wrapper = Wrapper(stream=stream)

start_date = '2017-01-04T00:00:00'
end_date = '2017-01-05T00:00:00'
dt_col = 'Settlement Day'

df = wrapper.query_API(start_date=start_date, end_date=end_date, dt_col=dt_col)

df.head(2)

,Settlement Period,Half-hourly Charge,Run Type,Total Daily BSUoS Charge,_full_text,BSUoS Price (£/MWh Hour),Settlement Day,_id
0,1,36104.6200000000026193447411060333251953125,II,1946056.1499999999068677425384521484375,"'-01':2 '-04':3 '00':5,6 '1':7 '1.68113':8 '19...",1.68113,2017-01-04T00:00:00,1
1,2,36968.389999999999417923390865325927734375,II,1946056.1499999999068677425384521484375,"'-01':2 '-04':3 '00':5,6 '1.74179':8 '1946056....",1.74179,2017-01-04T00:00:00,2


In [44]:
stream = 'weekly-wind-availability'
wrapper = Wrapper(stream)

df = wrapper.query_API()

df.head()

,_id,BMU_ID,Week Number,MW
0,1,ABRBO-1,09W20,0
1,2,ABRBO-1,10W20,0
2,3,ABRBO-1,11W20,0
3,4,ABRBO-1,12W20,0
4,5,ABRBO-1,13W20,0


In [45]:
stream = 'embedded-wind-and-solar-forecasts'
wrapper = Wrapper(stream)

df = wrapper.query_API()

df.head()

,_id,DATE_GMT,TIME_GMT,SETTLEMENT_DATE,SETTLEMENT_PERIOD,EMBEDDED_WIND_FORECAST,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_FORECAST,EMBEDDED_SOLAR_CAPACITY
0,1,20200502,2130,2020-05-02T00:00:00,45,408,6527,0,13080
1,2,20200502,2200,2020-05-02T00:00:00,46,386,6527,0,13080
2,3,20200502,2230,2020-05-02T00:00:00,47,386,6527,0,13080
3,4,20200502,2300,2020-05-02T00:00:00,48,357,6527,0,13080
4,5,20200502,2330,2020-05-03T00:00:00,1,357,6527,0,13082
